<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-11-04 12:58:59


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','Breakout','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock', 'MedianPE']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    68
1    62
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     21
VR       5
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    120
0     10
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    25
0    24
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%Min'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-05 00:00:00+05:30,19546.0,18512.0,5.58,^NSEI,20192.35,16945.05,15.35,45.0
2023-10-12 00:00:00+05:30,19794.0,18546.0,6.73,^NSEI,20192.35,16945.05,16.81,55.0
2023-10-19 00:00:00+05:30,19625.0,18587.0,5.58,^NSEI,20192.35,16945.05,15.81,47.0
2023-10-27 00:00:00+05:30,19047.0,18613.0,2.33,^NSEI,20192.35,16945.05,12.41,35.0
2023-11-03 00:00:00+05:30,19231.0,18642.0,3.16,^NSEI,20192.35,16945.05,13.49,44.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [14]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
df_prospects['Dev%_PE'] = round((df_prospects['Curr_PE'] - df_prospects['MedianPE'])*100/df_prospects['MedianPE'],2)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close', 'BizCheck',	'FinCheck',	'TechCheck', 'MedianPE', 'Curr_PE'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [15]:
# v40 and v40n
df_tmp = df_prospects[df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,19.16,46.01,59.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-2.20,5.0
ABBOTINDIA.NS,2.83,19.32,53.0,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,NaN,8.0
AKZOINDIA.NS,0.64,19.11,43.0,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
ANGELONE.NS,86.47,186.03,85.0,44.00,47.10,V40N,ATH,1,NaN,1,M,FINANCE,14.93,0.0
ASTRAZEN.NS,25.29,49.42,55.0,30.90,23.00,V40N,ATH,1,NaN,1,M,PHARMA,-30.38,2.0
AXISBANK.NS,6.33,20.12,49.0,6.15,14.50,V40,ATH,1,NaN,1,H,BANKS,-58.47,5.0
BAJAJ-AUTO.NS,23.59,61.39,62.0,26.20,20.20,V40,ATH,1,NaN,1,H,AUTO,16.84,2.0
BAJAJHLDNG.NS,4.47,25.93,51.0,9.71,11.20,V40,ATH,1,NaN,0,H,FINANCE,8.59,7.0
BAJFINANCE.NS,9.01,35.07,36.0,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,-34.98,10.0


In [16]:
# all other prospects
df_tmp = df_prospects[~df_prospects['Criteria'].isin(['V40','V40N'])]
data_table.DataTable(df_tmp, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ACC.NS,-0.81,16.42,35.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,46.0
ADANIENT.NS,-1.77,86.93,35.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,87.0
ADANIGREEN.NS,-7.03,95.31,44.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,229.0
ADANIPORTS.NS,10.32,73.15,50.0,9.53,14.30,NaN,NaN,1,NaN,1,L,NaN,NaN,21.0
ALKYLAMINE.NS,-11.34,1.40,36.0,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,NaN,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VGUARD.NS,9.23,23.29,45.0,15.20,12.60,NaN,NaN,1,NaN,0,L,NaN,NaN,7.0
VOLTAS.NS,1.66,13.37,46.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,108.76,57.0
WSTCSTPAPR.NS,30.00,58.35,52.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,6.0


In [17]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
BLUESTARCO.NS,28.53,72.50,68.0,24.40,23.40,AR,BTT,1,NaN,1,L,AC,24.65,55.0
UPL.NS,-16.75,3.43,34.0,14.30,13.40,AR,BTT,1,NaN,0,L,NaN,NaN,49.0
PNB.NS,31.14,81.12,54.0,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,-16.27,27.0
AKZOINDIA.NS,0.64,19.11,43.0,34.70,26.10,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
UJJIVANSFB.NS,41.71,136.50,50.0,1000.00,1000.00,V40N,BTT,1,NaN,1,M,BANKS,5852.38,11.0
BAJFINANCE.NS,9.01,35.07,36.0,11.80,23.50,V40,BTT,1,NaN,1,H,FINANCE,-34.98,10.0
NMDC.NS,35.13,59.59,61.0,30.20,23.00,VR,BTT,1,NaN,0,L,MINING,54.75,9.0
ABBOTINDIA.NS,2.83,19.32,53.0,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,NaN,8.0
J&KBANK.NS,54.93,176.94,53.0,4.92,13.10,VR,BTT,1,NaN,1,L,BANKS,53.75,8.0


In [18]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
KANSAINER.NS,3.74,23.23,29.0,14.50,10.7,V40N,ATH,1,NaN,1,M,PAINTS,-27.01,43.0
VINATIORGA.NS,-6.76,1.24,32.0,30.40,22.6,V40N,ATH,1,NaN,1,M,CHEMICALS,-5.62,35.0
AKZOINDIA.NS,0.64,19.11,43.0,34.70,26.1,V40,BTT,1,NaN,1,H,NaN,NaN,19.0
ITC.NS,2.91,35.53,42.0,39.00,29.1,V40,ATH,1,NaN,1,H,FMCG,36.04,14.0
MARICO.NS,0.20,12.80,34.0,41.90,35.8,V40,ATH,1,NaN,1,H,FMCG,-8.16,11.0
ABBOTINDIA.NS,2.83,19.32,53.0,41.10,31.6,V40,BTT,1,NaN,1,H,PHARMA,NaN,8.0
ICICIBANK.NS,2.05,15.21,49.0,6.32,17.2,V40,ATH,1,NaN,1,H,BANKS,-32.64,6.0


In [19]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
INDIANB.NS,-20.00,77.73,48.0,4.55,11.80,NaN,NaN,1,NaN,1,L,NaN,5.14,9923.0
ADANIGREEN.NS,-7.03,95.31,44.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,229.0
BLUESTARCO.NS,28.53,72.50,68.0,24.40,23.40,AR,BTT,1,NaN,1,L,AC,24.65,55.0
KANSAINER.NS,3.74,23.23,29.0,14.50,10.70,V40N,ATH,1,NaN,1,M,PAINTS,-27.01,43.0
IGL.NS,-11.57,5.10,38.0,26.60,20.50,NaN,NaN,1,NaN,1,L,NaN,NaN,40.0
VINATIORGA.NS,-6.76,1.24,32.0,30.40,22.60,V40N,ATH,1,NaN,1,M,CHEMICALS,-5.62,35.0
PNB.NS,31.14,81.12,54.0,4.10,3.34,VR,BTT,1,NaN,1,L,BANKS,-16.27,27.0
TASTYBITE.NS,31.64,104.50,51.0,1000.00,1000.00,NaN,NaN,1,NaN,1,L,NaN,NaN,25.0
MUTHOOTFIN.NS,16.23,47.92,63.0,12.10,17.90,NaN,NaN,1,NaN,1,L,FINANCE,12.00,25.0


In [20]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
JCHAC.NS,-4.65,14.11,33.0,1000.0,1000.0,V40N,ATH,1,NaN,0,M,AC,1823.08,200.0
SFL.NS,-4.97,12.37,42.0,13.6,12.5,V40N,ATH,1,NaN,0,M,MISC,17.44,84.0
SYMPHONY.NS,-9.42,1.90,45.0,14.8,13.5,V40N,ATH,1,NaN,0,M,DURABLES,-8.57,73.0
LALPATHLAB.NS,21.64,46.52,65.0,18.2,15.0,V40N,ATH,1,NaN,0,M,HEALTHCARE,5.43,60.0
DREAMFOLKS.NS,2.35,48.07,60.0,76.8,60.2,AR,ATH,0,NaN,0,L,MISC,NaN,56.0
BLUESTARCO.NS,28.53,72.50,68.0,24.4,23.4,AR,BTT,1,NaN,1,L,AC,24.65,55.0
PFIZER.NS,1.23,13.03,43.0,27.2,19.8,V40,ATH,1,NaN,0,H,PHARMA,217.81,54.0
UPL.NS,-16.75,3.43,34.0,14.3,13.4,AR,BTT,1,NaN,0,L,NaN,NaN,49.0
TTKPRESTIG.NS,4.92,21.02,49.0,18.1,13.8,V40N,ATH,1,NaN,0,M,DURABLES,13.10,47.0


In [21]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
INDIANB.NS,-20.00,77.73,48.0,4.55,11.80,NaN,NaN,1,NaN,1,L,NaN,5.14,9923.0
YESBANK.NS,-3.24,11.53,44.0,4.94,1.99,NaN,NaN,1,NaN,0,L,NaN,NaN,1628.0
ADANIGREEN.NS,-7.03,95.31,44.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,229.0
JCHAC.NS,-4.65,14.11,33.0,1000.00,1000.00,V40N,ATH,1,NaN,0,M,AC,1823.08,200.0
BIOCON.NS,-7.61,13.64,30.0,5.92,4.78,NaN,NaN,1,NaN,0,L,NaN,NaN,114.0
ALKYLAMINE.NS,-11.34,1.40,36.0,41.30,41.40,NaN,NaN,0,NaN,0,L,CHEMICALS,NaN,112.0
ADANIENT.NS,-1.77,86.93,35.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,87.0
SFL.NS,-4.97,12.37,42.0,13.60,12.50,V40N,ATH,1,NaN,0,M,MISC,17.44,84.0
SYMPHONY.NS,-9.42,1.90,45.0,14.80,13.50,V40N,ATH,1,NaN,0,M,DURABLES,-8.57,73.0


In [22]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 8


,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
ANGELONE.NS,86.47,186.03,85.0,44.00,47.1,V40N,ATH,1,NaN,1,M,FINANCE,14.93,0.0
COLPAL.NS,22.45,47.81,57.0,179.00,158.0,V40,BTT,1,NaN,1,H,FMCG,218.18,1.0
MCDOWELL-N.NS,20.22,45.61,62.0,20.00,17.5,V40N,ATH,1,NaN,1,M,BREWERIES,-15.17,1.0
ASTRAZEN.NS,25.29,49.42,55.0,30.90,23.0,V40N,ATH,1,NaN,1,M,PHARMA,-30.38,2.0
BAJAJ-AUTO.NS,23.59,61.39,62.0,26.20,20.2,V40,ATH,1,NaN,1,H,AUTO,16.84,2.0
EQUITASBNK.NS,23.80,97.78,59.0,8.04,12.2,V40N,ATH,1,NaN,1,M,BANKS,-16.09,2.0
HCLTECH.NS,12.56,28.79,56.0,28.30,23.0,V40,ATH,1,NaN,1,H,IT,36.81,2.0
TITAN.NS,15.95,42.33,57.0,25.10,30.8,V40,ATH,1,NaN,1,H,JEWELLERY,12.82,2.0


In [23]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%Min,RSI_14,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,Breakout,StarStock,Conviction,Category,Dev%_PE,Upside%Target
Stock,,,,,,,,,,,,,,
3MINDIA.NS,19.16,46.01,59.0,31.40,23.40,V40N,ATH,1,NaN,1,M,MISC,-2.20,5.0
ABBOTINDIA.NS,2.83,19.32,53.0,41.10,31.60,V40,BTT,1,NaN,1,H,PHARMA,NaN,8.0
ACC.NS,-0.81,16.42,35.0,10.00,7.06,NaN,NaN,0,NaN,0,L,CEMENT,NaN,46.0
ADANIENT.NS,-1.77,86.93,35.0,9.49,9.63,NaN,NaN,1,NaN,0,L,NaN,NaN,87.0
ADANIGREEN.NS,-7.03,95.31,44.0,7.81,22.80,NaN,NaN,1,NaN,1,L,NaN,NaN,229.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,-6.26,7.49,30.0,27.80,29.00,V40N,ATH,1,NaN,0,M,MISC,21.44,25.0
VOLTAS.NS,1.66,13.37,46.0,9.67,4.42,NaN,NaN,1,NaN,0,L,AC,108.76,57.0
WSTCSTPAPR.NS,30.00,58.35,52.0,52.90,44.80,VR,ATH,0,NaN,0,L,PAPER,NaN,6.0


In [24]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [25]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%Min,RSI_14
Date,,,,,,,,
2023-10-05 00:00:00+05:30,563.0,525.0,7.28,BERGEPAINT.NS,710.11,447.11,25.96,39.0
2023-10-12 00:00:00+05:30,566.0,527.0,7.37,BERGEPAINT.NS,710.11,447.11,26.56,42.0
2023-10-19 00:00:00+05:30,576.0,529.0,8.94,BERGEPAINT.NS,710.11,447.11,28.89,48.0
2023-10-27 00:00:00+05:30,538.0,531.0,1.23,BERGEPAINT.NS,710.11,447.11,20.23,32.0
2023-11-03 00:00:00+05:30,553.0,533.0,3.78,BERGEPAINT.NS,710.11,447.11,23.72,43.0
